In [5]:
!pip install -q openai langchain beautifulsoup4 chroma chromadb tiktoken langchainhub crewai "unstructured[csv]"
!pip install --upgrade --quiet  semanticscholar
!pip install --quiet xmltodict

In [2]:
#from getpass import getpass
#import os
#os.environ['OPENAI_API_KEY'] = getpass()

In [16]:
#Define tools
import re
import time

from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
#from openai import OpenAI
from CustomLLM import ARGO_LLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

client = ARGO_LLM() #OpenAI()

from langchain.tools import DuckDuckGoSearchRun
duck_search_tool = DuckDuckGoSearchRun()

from langchain_community.tools.pubmed.tool import PubmedQueryRun
pubmed_tool = PubmedQueryRun()

from langchain_community.tools.semanticscholar.tool import SemanticScholarQueryRun
scholar_tool = SemanticScholarQueryRun()

search_tools = [duck_search_tool,pubmed_tool,scholar_tool]

import zipfile
from langchain_community.document_loaders import DirectoryLoader
import glob
import unstructured
import os

# Step 1: Absolute path (replace with the actual absolute path)
absolute_path = './EpiHiper-Schema-master/'

# Step 2: Check if the directory exists
if not os.path.exists(absolute_path):
    print(f"Directory does not exist: {absolute_path}")
else:
    print(f"Directory exists: {absolute_path}")

    # Step 3: Manually list files
    file_paths = glob.glob(absolute_path + '/**/*.*', recursive=True)
    print(f"Manually found {len(file_paths)} files:")
    #for file_path in file_paths:
    #    print(file_path)
    file_contents_dict={}
    for file_path in file_paths:
        try:
            filename = os.path.basename(file_path)
            with open(file_path, 'r') as file_pt:
                content = file_pt.read()
            if filename in file_contents_dict:
                file_contents_dict[filename] += '\n' + content
            else:
                file_contents_dict[filename] = content
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")

#loader2 = DirectoryLoader('./EpiHiper-Schema-master/', glob="**/*.*", show_progress=True)
#abm_codes = loader2.load()
# Debugging: Print the number of loaded files
#print(f"Number of loaded files: {len(abm_codes)}")

#code_splits = split_by_length_with_overlap(abm_codes)
#vectorstore = Chroma.from_documents(documents=abm_codes, embedding=OpenAIEmbeddings())
#ABM_retriever = vectorstore.as_retriever()

disease_model_schema = file_contents_dict['diseaseModelSchema.json']
disease_model_rules = file_contents_dict['diseaseModelRules.json']

Directory exists: ./EpiHiper-Schema-master/
Manually found 118 files:


In [19]:
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process
import textwrap

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=.2)
llm = ARGO_LLM() #"gpt-4-1106-preview", "gpt-4-0314"

PRISMA_expert = Agent(
    role="PRISMA Meta-Analysis Expert",
    goal="Ensure any meta-analysis of papers and data are performed according to the PRISMA guidelines for meta-analyses",
    backstory=textwrap.dedent("""
    Created by a collaborative effort of research institutions worldwide, PRISMA_expert embodies the collective 
    wisdom of leading experts in systematic reviews and meta-analysis. Its inception was motivated by the need 
    for rigorous, transparent, and replicable research findings in the scientific community. With the PRISMA 
    (Preferred Reporting Items for Systematic Reviews and Meta-Analyses) guidelines at its core, PRISMA_expert 
    is designed to assist researchers in conducting meta-analyses that are both comprehensive and methodologically 
    sound. It leverages advanced algorithms to analyze research data, identify relevant studies, and assess 
    their quality, ensuring that the results of meta-analyses are reliable and actionable. PRISMA_expert also 
    educates users on best practices for systematic reviews, aiming to elevate the quality of research across 
    disciplines. Its development was fueled by the increasing complexity of research data and the challenges 
    associated with synthesizing vast amounts of scientific information. By streamlining the meta-analysis 
    process and enforcing adherence to PRISMA guidelines, PRISMA_expert aims to facilitate the generation of 
    evidence-based insights, thereby advancing scientific knowledge and informing policy decisions.
    """),
#    backstory=textwrap.dedent("""
#        PRISMA_expert, whose real name is Dr. Emily Harris, is a distinguished researcher and academic 
#        in the field of evidence-based medicine. Her journey into becoming a PRISMA Meta-Analysis Expert 
#        began during her early years as a medical student. Dr. Harris was always intrigued by the power 
#        of systematic reviews and meta-analyses in synthesizing existing research to inform clinical 
#        decision-making. She saw the potential for improving healthcare outcomes by ensuring that these 
#        analyses were conducted rigorously and transparently.
#
#        As a student, Emily Harris struggled to find comprehensive resources on how to conduct 
#        meta-analyses properly. She was often frustrated by the lack of clear guidelines and standards 
#        in the field. Determined to make a difference, she decided to pursue a career dedicated to 
#        rectifying this issue. After completing her medical degree, Emily embarked on a PhD program in 
#        epidemiology and evidence-based medicine, focusing on meta-analysis methodology.
#
#        During her academic journey, Dr. Harris published several groundbreaking papers on meta-analysis 
#        techniques and guidelines. Her dedication and expertise caught the attention of the PRISMA Group, 
#        a renowned organization that develops and maintains the PRISMA (Preferred Reporting Items for 
#        Systematic Reviews and Meta-Analyses) guidelines. Impressed by her work, they invited her to join 
#        their expert panel.
#
#        Emily gladly accepted the invitation and became an integral part of the PRISMA Group. Over the 
#        years, she played a pivotal role in refining and updating the PRISMA guidelines, ensuring they 
#        stayed aligned with the evolving landscape of evidence synthesis. Dr. Harris's work not only 
#        contributed to the improvement of meta-analysis standards but also helped researchers worldwide 
#        conduct more reliable and transparent reviews.
#
#        Today, as a PRISMA Meta-Analysis Expert, Dr. Emily Harris is on a mission to ensure that any 
#        meta-analysis of papers and data adheres to the PRISMA guidelines. She imparts her knowledge 
#        and expertise through workshops, webinars, and consultations with research teams. Her ultimate 
#        goal is to raise the quality and credibility of meta-analyses across various fields of research, 
#        thus advancing evidence-based decision-making in healthcare and beyond.
#    """),
    verbose=True,
    allow_delegation=False,
    tools=[],  ###
    llm=llm,
)

query_executor = Agent(
    role="""
        Agent Role: Information Gathering and Analysis Specialist
    """,
    goal=textwrap.dedent("""
        To efficiently retrieve, analyze, and synthesize information from diverse sources to answer 
        complex queries and support decision-making processes. The agent aims to leverage its integrated 
        tools to access a wide range of databases and search engines, including scientific literature, 
        general knowledge, and academic research, ensuring comprehensive and accurate responses.
    """),
    backstory=textwrap.dedent("""
        Developed by a consortium of data scientists and information technology experts, Query_Executor 
        is the culmination of efforts to bridge the gap between vast information repositories and the 
        need for actionable intelligence. With the digital age flooding every sector with data, the 
        ability to sift through, analyze, and present this information in a coherent and useful manner 
        has become invaluable. Query_Executor integrates cutting-edge natural language processing 
        capabilities with advanced search technologies from DuckDuckGo, PubMed, and Semantic Scholar. 
        This allows it to navigate through millions of data points, from academic papers to general 
        knowledge, efficiently and effectively. Its inception was driven by the demand for a tool that 
        could support researchers, analysts, and decision-makers by providing quick access to relevant 
        information and insights, helping to make informed decisions in a timely manner. By synthesizing 
        information from both broad and specialized databases, Query_Executor stands as a versatile and 
        powerful assistant in research, planning, and strategic analysis across various fields.
    """),
    verbose=True,
    llm=llm,
    # tools=[SqlTools.do_sql_query, RagTools.do_rag_query],
    tools=search_tools,
    allow_delegation=True,
)

param_executor = Agent(
    role="""
        Agent Role: Model Parameterizer
        """,
    goal="Plan the steps needed to parameterize an agent-based simulation",
    backstory=textwrap.dedent("""
                You are an expert at identifying modeling parameters from code base that implements 
        an agent-based model and listing the model choices, parameters, and json config files 
        whose values need to be determined. You will break down each model choice, parameter, 
        and json config file into sub-questions such that the answer to each sub-question will 
        inform the value to be used in the agent-based simulation.
        Accept the user-question and determine if it requires sub-questions to either the
        CDC website which provides an official source of recent infectious disease outbreaks
        or Wikipedia for information about a geographical location, country, infectious agent
        characteristics, transmission dynamics, infection states, or other epidemiological
        modeling efforts.
        Your final answer MUST be a description of sub-questions and step-by-step instructions
        that explain how to find the data needed to choose the best model parameter ranges 
        for an agent-based modeling code base.
    """),
    verbose=True,
    llm=llm,
    # tools=[SqlTools.do_sql_query, RagTools.do_rag_query],
    tools=[],
    allow_delegation=True,
)

In [22]:
user_query = "Model the current flavivirus outbreak using an agent based model"

task1 = Task(
    description=textwrap.dedent(f"""
        Your task is to go through a json file in the agent-based model code base understand
        what data is needed in order to parameterize the below schema:
            {disease_model_schema}
        Decide what general disease information and questions would be useful in light of the
        below user query: 
            {user_query}
        The final output should be a list of questions and search strategies.
    """),
    agent=param_executor
)

task2 = Task(
    description=textwrap.dedent(f"""
        You will recieve a set of queries and information from the previous task. Your task is
        to gather the information needed and to summarize the answers to the list of sub-questions
        based on that information.
    """),
    agent=query_executor
)

task3 = Task(
    description=textwrap.dedent(f"""
        You will recieve a set of queries and information from the previous task. You will:
        1. Examine the information, critique it, and decide if the numbers are numerically reasonable 
        in light of the data provided for the meta-analysis.
        2. If absolute numbers are not available from in the provided data, then you will 
        examine the relative values of the field values to see if they are related appropriately,
        i.e., greater than, less than, or approximately equal. 
        3. You will list field values that should be altered from points 1 and 2, their original
        value, and suggest a new value.
        Your final answer must a range of values for all the parameters specified in the JSON
        file.
    """),
    agent=PRISMA_expert
)

In [23]:
crew = Crew(
    agents=[param_executor,PRISMA_expert,query_executor],
    tasks=[task1,task2,task3],
    verbose=2,  # print what tasks are being worked on, can set it to 1 or 2
    process=Process.sequential,
)

result = crew.kickoff()

print("######################")
print(result)

Working Agent: 
        Agent Role: Model Parameterizer
        
Starting Task: 
        Your task is to go through a json file in the agent-based model code base understand
        what data is needed in order to parameterize the below schema:
            [{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "$id": "diseaseModel",
  "title": "Disease Model",
  "description": "The disease model describes the possible disease state changes within an individual including transmission.",
  "type": "object",
  "required": [
    "epiHiperSchema",
    "states",
    "initialState",
    "transmissions",
    "transitions"
  ],
  "properties": {
    "epiHiperSchema": {
      "type": "string",
      "pattern": "^.*/diseaseModelSchema.json$"
    },
    "states": {
      "type": "array",
      "description": "An array of states",
      "items": {
        "title": "state",
        "$$target": [
          "#/properties/states/items"
        ],
        "description": "A state in the contagion m

ValueError: stop kwargs are not permitted.

In [302]:
#from langchain import hub
#prompt = hub.pull("hwchase17/self-ask-with-search")
#print(f'{prompt.format(agent_scratchpad = "AGENTSCRATCHPAD", input = "INPUT")}')

from langchain.tools.retriever import create_retriever_tool

CDC_retriever_tool = create_retriever_tool(
    CDC_retriever,
    "CDC_retriever_tool",
    """As an AI assistant you provide answers based on the given context, ensuring accuracy and briefness. 

        You always follow these guidelines:

        -If the answer isn't available within the context, state that fact
        -Otherwise, answer to your best capability, refering to source of documents provided
        -Only use examples if explicitly requested
        -Do not introduce examples outside of the context
        -Do not answer if context is absent
        -Limit responses to three or four sentences for clarity and conciseness
        
        Search for data related to outbreaks. For questions about outbreaks, use this tool to return 
        relevant data for answering questions about outbreaks""",
)

ABM_retriever_tool = create_retriever_tool(
    ABM_retriever,
    "ABM_retriever_tool",
    """As an AI assistant you provide answers based on the given context, ensuring accuracy and briefness. 

        You always follow these guidelines:

        -If the answer isn't available within the context, state that fact
        -Otherwise, answer to your best capability, refering to source of documents provided
        -Only use examples if explicitly requested
        -Do not introduce examples outside of the context
        -Do not answer if context is absent
        -Limit responses to three or four sentences for clarity and conciseness
        
        Search for data related to outbreaks. For questions about outbreaks, use this tool to return 
        relevant data for answering questions about outbreaks""",
)


critic = Agent(
    role="Evaluate Answer",
    goal="Provide feedback on prior responses to user query",
    backstory=textwrap.dedent("""
        You are an expert at understanding, correcting, and producing json formatted strings.
        You know how to list the model choices and parameters in a json schema file format and
        how to convert text about model choices and parameters into a json schema, following
        any examples provided.
        You will provide critical feedback and improve the final output using that feedback.
        Your final answer MUST be a json schema file for an agent-based modeling code base.
    """),
    verbose=True,
    allow_delegation=False,
    tools=[],  ###
    llm=llm,
)

In [303]:
param_executor_old = Agent(
    role="""
        Agent Role: Model Parameterizer

        Primary Objectives:
        1. Identify the json schema in the context and assess the information needed to assign values to all json fields.

        2. Examine the context for additional information from prior prompts and searches and use those to assign value to each json field.

        3. If there is any missing information that is needed, ask very specific questions and use CDC_retriever_tool or web_search to find the needed information.

        4. Produce a json file in the same format as the json schema with the field values filled in according to the information provided.

        Key Responsibilities:
        - Make use of information provided
        - Synthesize information from diverse sources to provide a comprehensive understanding of the disease and its impact.
        - Adhere to the json format in your final output.
        """,
    goal="Parameterize an agent-based model",
    backstory=textwrap.dedent("""
        Take in a json schema file and output an updated json file with values filled in based on information provided
    """),
    verbose=True,
    llm=llm,
    # tools=[SqlTools.do_sql_query, RagTools.do_rag_query],
    tools=[CDC_retriever_tool, web_search],
    allow_delegation=True,
)

json_validator_old = Agent(
    role="JSON format validator",
    goal="Make sure the final answer is in the appropriate json format",
    backstory=textwrap.dedent("""
        You are an expert at understanding, correcting, and producing json formatted strings.
        You know how to list the model choices and parameters in a json schema file format and
        how to convert text about model choices and parameters into a json schema, following
        any examples provided. 
        Your final answer MUST be a json schema file for an agent-based modeling code base.
    """),
    verbose=True,
    allow_delegation=False,
    tools=[],  ###
    llm=llm,
)

task1 = Task(
    description=textwrap.dedent(f"""
        Your task is to go through a json file in the agent-based model code base and set the 
        values for each field that needs to be parameterized in the below schema:
            {disease_model_schema}
        Use the CDC_retriever_tool data about current outbreaks and web_search find general 
        disease information and characteristics and produce a json output with the values 
        needed to carry out the below user query: 
            {user_query}
        The final output should be a json formatted string.
    """),
    agent=param_executor
)

task2 = Task(
    description=textwrap.dedent(f"""
        You will recieve a set of queries and information from the previous task. Your task is
        make sure the information gathered conform to the json schema format provided.
        Your final answer must be be a json formatted string.
    """),
    agent=json_validator
)

task2b = Task(
    description=textwrap.dedent(f"""
        Your task is to go through a json file in the agent-based model code base and set the 
        values for each field that needs to be parameterized in the below schema:
            {disease_model_schema}
        Use the CDC_retriever_tool data about current outbreaks and web_search find general 
        disease information and characteristics and produce a json output with the values 
        needed to carry out the below user query: 
            {user_query}
        The final output should be a json formatted string.
    """),
    agent=param_executor
)

#Example that got me to JSON param file #1
crew = Crew(
    agents=[param_executor_old,json_validator_old],
    tasks=[task1,task2],
    verbose=2,  # print what tasks are being worked on, can set it to 1 or 2
    process=Process.sequential,
)

result = crew.kickoff()

print("######################")
print(result)


Working Agent: 
        Agent Role: Model Parameterizer

        Primary Objectives:
        1. Identify the json schema in the context and assess the information needed to assign values to all json fields.

        2. Examine the context for additional information from prior prompts and searches and use those to assign value to each json field.

        3. Produce a json file in the same format as the json schema with the field values filled in according to the information provided.

        Key Responsibilities:
        - Make use of information provided
        - Synthesize information from diverse sources to provide a comprehensive understanding of the disease and its impact.
        - Adhere to the json format in your final output.
        
Starting Task: 
        Your task is to go through a json file in the agent-based model code base and set the 
        values for each field that needs to be parameterized in the below schema:
            [{
  "$schema": "http://json-schema.org

In [304]:
user_query = "Model the current flavivirus outbreak using an agent based model"

planning_task = Task(
    description=textwrap.dedent(f"""
        Your task is to plan out the necessary steps in order to fulfull the user task and 
        create smaller prompts for other co-workers to follow. The result of this task should
        be an organized list of questions to be answered in order to fulfill the user request.
            {user_query}
        Use the following json template that comes from an agent-based model code base to help
        you decide what subtasks need to be completed to fulfull the user request:
            {disease_model_schema}
    """),
    agent=query_planner
)

list_steps_task = Task(
    description=textwrap.dedent(f"""
        You will recieve a set of subquestions from the previous task. Your task is to format
        them as a list of steps saved as a python list of strings.
    """),
    agent=step_planner
)

critic_task = Task(
    description=textwrap.dedent(f"""
        You will recieve a set of queries and information from the previous task. You will:
        1. Examine the JSON output and critique it based on information provided, decide if the 
        numbers are numerically reasonable in light of the provided data.
        2. If absolute numbers are not available from in the provided data, then you will 
        examine the relative values of the field values to see if they are related appropriately,
        i.e., greater than, less than, or approximately equal. 
        3. You will list field values that should be altered from points 1 and 2, their original
        value, and suggest a new value.
        4. You will incorporate those values into the full JSON schema format and check for proper 
        JSON formatting and make any corrections needed.
        5. Your final answer MUT be be a json formatted string.
    """),
    agent=json_validator
)


In [316]:
crew = Crew(
    agents=[query_planner,step_planner],
    tasks=[planning_task,list_steps_task],
    verbose=2,  # print what tasks are being worked on, can set it to 1 or 2
    process=Process.sequential,
)

result = crew.kickoff()

print("######################")
print(result)

#get output of list_steps_task into a python list
test = result.replace('\n','').replace("`",'').replace("  ",'')
test = re.sub(r"\].*", "", test) + "]"
test = "[" + re.sub(r"^.*?\[", "", test)
import ast
test2= ast.literal_eval(test)

for q in test2:
    print(q)

answers = []
for q in test2:
    subquestion_task = Task(
        description=textwrap.dedent(f"""
            Your task is to find the information needed to answer the question:
                {q}
            In the context of the original user request:
                {user_query}
        """),
        agent=query_executor
    )
    crew = Crew(
        agents=[query_executor],
        tasks=[subquestion_task],
        verbose=2,  # print what tasks are being worked on, can set it to 1 or 2
        process=Process.sequential,
    )

    answer = crew.kickoff()
    answers.append(answer)

print("######################")
print(answers)

answers_dump = '\n'.join(answers)

compile_params_task = Task(
    description=textwrap.dedent(f"""
        USEFUL INFORMATION:
            {answers_dump}
        Your task is to go through a json file in the agent-based model code base and set the 
        values for each field that needs to be parameterized in the below schema:
            {disease_model_schema}
        Use the USEFUL INFORMATION to produce a json output with the values 
        needed to carry out the below user query: 
            {user_query}
        The final output should be a json formatted string.
    """),
    agent=param_executor
)

crew = Crew(
    agents=[param_executor,critic],
    tasks=[compile_params_task,critic_task],
    verbose=2,  # print what tasks are being worked on, can set it to 1 or 2
    process=Process.sequential,
)

result2 = crew.kickoff()

print("######################")
print(result2)



Working Agent: Simulation Planner
Starting Task: 
        Your task is to plan out the necessary steps in order to fulfull the user task and 
        create smaller prompts for other co-workers to follow. The result of this task should
        be an organized list of questions to be answered in order to fulfill the user request.
            Model the current flavivirus outbreak using an agent based model
        Use the following json template that comes from an agent-based model code base to help
        you decide what subtasks need to be completed to fulfull the user request:
            [{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "$id": "diseaseModel",
  "title": "Disease Model",
  "description": "The disease model describes the possible disease state changes within an individual including transmission.",
  "type": "object",
  "required": [
    "epiHiperSchema",
    "states",
    "initialState",
    "transmissions",
    "transitions"
  ],
  "properties": {
    "epi

In [292]:
param_executor2 = Agent(
    role="""
        Agent Role: Model Parameterizer

        Primary Objectives:
        1. Identify the json schema in the context and assess the information needed to assign values to all json fields.

        2. Examine the context for additional information from prior prompts and searches and use those to assign value to each json field.

        3. Produce a json file in the same format as the json schema with the field values filled in according to the information provided.

        Key Responsibilities:
        - Make use of information provided
        - Synthesize information from diverse sources to provide a comprehensive understanding of the disease and its impact.
        - Adhere to the json format in your final output.
        """,
    goal="Parameterize an agent-based model",
    backstory=textwrap.dedent("""
        Take in a json schema file and output an updated json file with values filled in based on information provided
    """),
    verbose=True,
    llm=llm,
    # tools=[SqlTools.do_sql_query, RagTools.do_rag_query],
    tools=[CDC_retriever_tool, web_search],
    allow_delegation=True,
)

task5 = Task(
    description=textwrap.dedent(f"""
        USEFUL INFORMATION:
            {answers_dump}
        Your task is to go through a json file in the agent-based model code base and set the 
        values for each field that needs to be parameterized in the below schema:
            {disease_model_schema}
        Produce a modified json schema with the values needed to carry out the below user request: 
            {user_query}
        The final output should be a json formatted schema that has been updated to different 
        field values than the initial example to incorporate as much information as possible
        about the disease mentioned in the user request. When uncertain, try to use estimate
        values.
    """),
    agent=param_executor
)

crew = Crew(
    agents=[param_executor2,critic],
    tasks=[task5,task2b],
    verbose=2,  # print what tasks are being worked on, can set it to 1 or 2
    process=Process.sequential,
)

result2 = crew.kickoff()

print("######################")
print(result2)

Working Agent: 
        Agent Role: Model Parameterizer

        Primary Objectives:
        1. Identify the json schema in the context and assess the information needed to assign values to all json fields.

        2. Examine the context for additional information from prior prompts and searches and use those to assign value to each json field.

        3. If there is any missing information that is needed, ask very specific questions and use CDC_retriever_tool or web_search to find the needed information.

        4. Produce a json file in the same format as the json schema with the field values filled in according to the information provided.

        Key Responsibilities:
        - Make use of information provided
        - Synthesize information from diverse sources to provide a comprehensive understanding of the disease and its impact.
        - Adhere to the json format in your final output.
        
Starting Task: 
        USEFUL INFORMATION:
            The incubation period f

In [312]:
test

'steps = ["What is the current flavivirus being modeled (e.g., Zika, Dengue, Yellow Fever)?","What are the known states of infection for this flavivirus (e.g., susceptible, exposed, infectious, recovered)?","What is the initial state of the population regarding the flavivirus infection?","What are the transmission rates of the flavivirus?","What are the probabilities of transitioning from one state to another (e.g., from exposed to infectious)?","What is the average dwell time in each state of infection?","Are there any known factors that affect susceptibility or infectivity (e.g., age, pre-existing immunity, vector density)?","What is the geographical location of the outbreak?","What is the population size and density in the affected area?","What are the demographics of the population (e.g., age distribution, vaccination status)?","What is the current number of reported cases and fatalities?","What is the estimated R0 (basic reproduction number) for the outbreak?","Are there any contr

In [314]:
test

'steps = ["Identify the specific flavivirus causing the outbreak: What is the name of the flavivirus causing the current outbreak? What are the characteristics of this flavivirus (e.g., incubation period, symptoms, mortality rate)?","Determine the disease states and transitions: What are the possible states an individual can be in regarding the flavivirus infection (e.g., susceptible, exposed, infectious, recovered)? What are the transition probabilities between these states? What is the average dwell time in each state?","Define the initial state of the simulation: What is the initial number of individuals in each disease state at the start of the simulation? What is the initial distribution of the population in terms of age, sex, and other relevant demographic factors?","Configure the transmission dynamics: What is the mode of transmission of the flavivirus (e.g., vector-borne, direct contact)? What is the basic reproduction number (R0) of the virus? How does the transmissibility cha

In [315]:
test2

['Identify the specific flaviviruses involved in the current outbreak.',
 'Determine the known states of infection for the specific flavivirus (e.g., susceptible, exposed, infectious, recovered).',
 'Assess the initial state of the population regarding the flavivirus infection.',
 'Calculate the transmission rates of the flavivirus within the population.',
 'Establish the recovery rates and mortality rates associated with the flavivirus.',
 'Define the unique identifiers for each state of infection.',
 'Evaluate the susceptibility of individuals in each state to the flavivirus.',
 'Measure the infectivity of individuals in each state.',
 'Determine the unique identifier for the initial state of the population.',
 'Identify the unique identifiers for each type of transmission event.',
 'Specify the entry and exit states for each transmission event.',
 'Identify the contact state that leads to transmission.',
 'Calculate the transmissibility factor for each transmission event.',
 'Consid